# First Practical Work

## Data Science and Engineering

### DESIGN OF AN IMAGE FILTER FUNCTION, PARALLELIZABLE AND SCALABLE

## Authors:

**Full name:** Mireia Alba Kesti Izquierdo 
    
**NIA**:** 100406960

**Full name:** Aleksandra Jamróz
    
**NIA**:** 100491363

Python multiprocessing documentation https://docs.python.org/3/library/multiprocessing.html

In [ ]:
import numpy as np
import multiprocessing as mp
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cProfile
import ctypes

In [ ]:
# loading our own module
import myfunctions as my
import importlib
importlib.reload(my)

### Image preparation 

First we have to take care about our image. Choose the file, open it, convert to numpy array and check the dimentions.

In [ ]:
# defining the images for filtering
chess_file = "chess.jpg"
digits_file = "digits.jpg"
fence_file = "fence.jpg"
hand_x_ray_file = "hand-x-ray.jpg"
jupiter_file = "cloudsonjupi.jpg"

# opening chosen file
F_IMAGE = Image.open(chess_file)

# defining cmap (default is color image, if it's grayscale, cmap is changed later)
cmap = None

In [ ]:
# get some information about the image
print(F_IMAGE.format)
print(F_IMAGE.size)
print(F_IMAGE.mode)

In [ ]:
# load the image & convert it to numpy array
image = np.array(F_IMAGE)

# check that the image was converted into numpy array correctly
print(type(image))

In [ ]:
# checking if image is gray-scale, changing cmap and expanding one dimention if necessary
if len(image.shape) == 2:
    cmap = "gray"
    image = np.expand_dims(image, axis=2)

In [ ]:
# visualise the picture
plt.figure()
plt.imshow(image, cmap=cmap)

In [ ]:
# get the value of each pixel of the numpy array image
print(image)

### Definitions of filters
* The first filter is impulse response filter (the image output must be equal to the original one).
* The second filter is an edge filter, first order in x axis,  
* The third filter is an edge filter, first order in y axis,
* the fourth filter is an edge filter, second order, bi-directional
* the fifth filter is a blur gausian filter.

In [ ]:
# definitions of 5 available filters as numpy arrays

filter1 = np.array([[0,0,0,0,0],
                    [0,0,0,0,0],
                    [0,0,1,0,0],
                    [0,0,0,0,0],
                    [0,0,0,0,0]])

filter2 = np.array([[0.5, 0 , -0.5]])

filter3 = np.array([[0.5],[0],[-0.5]])

filter4 = np.array([[1,0,-1],
                    [2,0,-2],
                    [1,0,-1]])

filter5 = np.array([[0.00078633,0.00655965,0.01330373,0.00655965,0.00078633],
                    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
                    [0.01330373,0.11098164,0.22508352,0.11098164,0.01330373],
                    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
                    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633]])

### Preparation for filtering

In order to filter the image, we have to define some variables: how many processes we want to start, how much space will each of filtered image take and then define shared spaces in which our filtered images will be stored.

In [ ]:
# setting the number of processes
NUMPROCESS = 8

In [ ]:
# calculating data size to allocate in memory for filtered image
data_buffer_size = image.shape[0] * image.shape[1] * image.shape[2]
print("Size of the image: ", data_buffer_size)


In [ ]:
# defining multiprocessing vectors (shared spaces)
filtered_image1_VECTOR = mp.Array(ctypes.c_byte, data_buffer_size)
filtered_image2_VECTOR = mp.Array(ctypes.c_byte, data_buffer_size)

### Applying 1 filter

Let's check if filtering function implemented in first separate module works correctly. We will run it on an image with one filter and see the result.

In [ ]:
my.image_filter(image, filter2, NUMPROCESS, filtered_image1_VECTOR)  

In [ ]:
filtered_image1 = my.tonumpyarray(filtered_image1_VECTOR).reshape(image.shape)

In [ ]:
plt.figure()
plt.imshow(filtered_image1, cmap=cmap)

As we see, it works correctly. Now we can move to the second part.

### Applying 2 filters

Here you can find a function which filters 1 image with 2 filters at the same time. Filtering functions are in the separate modules in order to create  separate memory spaces. Definitions of them can be found in myfunctions module and myfunctions2 file.

In [ ]:
def filters_execution(image: np.array,  
                      filter_mask1: np.array, 
                      filter_mask2: np.array,  
                      numprocessors: int,
                      filtered_image1: mp.Array,
                      filtered_image2: mp.Array ):
    """
    Function invoking 2 different parallel processes, each executing a filter
    on the same image and saving the result to independent memory spaces. Uses
    previous function for filtering.
    """

    # creating two processes
    p1 = mp.Process(target = my.image_filter, args = (image, filter_mask1, numprocessors, filtered_image1))
    p2 = mp.Process(target = my.image_filter, args = (image, filter_mask2, numprocessors, filtered_image2))
    
    # starting processes 
    p1.start() 
    p2.start() 
  
    # wait until processes are finished 
    p1.join() 
    p2.join()

    return

In [ ]:
# Filters execution with chosen filters
filters_execution(image, filter3, filter4, NUMPROCESS, filtered_image1_VECTOR, filtered_image2_VECTOR)

### Visualization
Here we take the filtered images stored in the multiprocessing.Vector variables filtered_image1_VECTOR and filtered_image2_VECTOR, convert to numpy array, with the same shape of the orginal image, and show the results.

In [ ]:
filtered_image1 = my.tonumpyarray(filtered_image1_VECTOR).reshape(image.shape)
filtered_image2 = my.tonumpyarray(filtered_image2_VECTOR).reshape(image.shape)

In [ ]:
plt.figure()
plt.imshow(filtered_image1, cmap=cmap)

In [ ]:
plt.figure()
plt.imshow(filtered_image2, cmap=cmap)